In [1]:
import os # импортировать библиотеку для работы с файлами

In [2]:
%pwd # получить текущую директорию

'c:\\Users\\Mikhnin\\Desktop\\project_1\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../") # перейти в директорию выше

In [4]:
%pwd # получить текущую директорию

'c:\\Users\\Mikhnin\\Desktop\\project_1\\Text-Summarizer-Project'

Update entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

Update the configuration manager in src config


In [6]:
from textSummarizer.constants import * # импортировать константы
from textSummarizer.utils.common import read_yaml, create_directories # импортировать функции

In [7]:
class ConfigurationManager: # создать класс для работы с конфигурацией
    def __init__( # создать конструктор класса
        self,
        config_filepath = CONFIG_FILE_PATH, # задать путь к файлу конфигурации
        params_filepath = PARAMS_FILE_PATH): # задать путь к файлу параметров

        self.config = read_yaml(config_filepath) # прочитать файл конфигурации
        self.params = read_yaml(params_filepath) # прочитать файл параметров

        create_directories([self.config.artifacts_root])    # создать директорию для артефактов


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig: # создать функцию для получения конфигурации
        config = self.config.model_evaluation # получить конфигурацию

        create_directories([config.root_dir])   # создать директорию для артефактов

        model_evaluation_config = ModelEvaluationConfig( # создать конфигурацию
            root_dir=config.root_dir, # задать корневую директорию
            data_path=config.data_path, # задать путь к данным
            model_path = config.model_path, # задать путь к модели
            tokenizer_path = config.tokenizer_path, # задать путь к токенизатору
            metric_file_name = config.metric_file_name # задать имя файла с метриками
           
        ) 

        return model_evaluation_config # вернуть конфигурацию

Update the conponents


In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer 
# импортировать классы для работы с моделью и токенизатором
from datasets import load_dataset, load_from_disk, load_metric 
# импортировать классы для работы с датасетом и метриками
import torch # импортировать библиотеку для работы с тензорами
import pandas as pd # импортировать библиотеку для работы с таблицами
from tqdm import tqdm # импортировать класс для отображения прогресса

In [10]:
class ModelEvaluation: # создать класс для оценки модели
    def __init__(self, config: ModelEvaluationConfig): # создать конструктор класса
        self.config = config # задать конфигурацию


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size): 
        # создать функцию для разделения датасета на батчи
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size): # создать цикл
            yield list_of_elements[i : i + batch_size] # вернуть батч

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer,  # создать функцию для вычисления метрики
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",  # задать параметры функции
                               column_text="article",  # задать параметры функции
                               column_summary="highlights"): # задать параметры функции
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size)) # разделить датасет на батчи
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size)) # разделить датасет на батчи

        for article_batch, target_batch in tqdm( # создать цикл
            zip(article_batches, target_batches), total=len(article_batches)): # создать цикл
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, # токенизировать текст
                            padding="max_length", return_tensors="pt") # токенизировать текст
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device), # сгенерировать суммаризации
                            attention_mask=inputs["attention_mask"].to(device),  # сгенерировать суммаризации
                            length_penalty=0.8, num_beams=8, max_length=128) # сгенерировать суммаризации
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,  # декодировать суммаризации
                                    clean_up_tokenization_spaces=True)  # декодировать суммаризации
                for s in summaries]      # декодировать суммаризации
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries] # декодировать суммаризации
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch) # добавить батч в метрику
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute() # вычислить метрику
        return score # вернуть метрику


    def evaluate(self): # создать функцию для оценки модели
        device = "cuda" if torch.cuda.is_available() else "cpu" # задать устройство для обучения
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path) # загрузить токенизатор
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device) # загрузить модель
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path) # загрузить датасет


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"] # задать метрики
  
        rouge_metric = load_metric('rouge') # загрузить метрику

        score = self.calculate_metric_on_test_ds( # вычислить метрику
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, 
        batch_size = 2, column_text = 'dialogue', column_summary= 'summary' # задать параметры функции
            )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names ) # создать словарь с метриками

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] ) # создать таблицу с метриками
        df.to_csv(self.config.metric_file_name, index=False) # сохранить таблицу с метриками


Update the pipeline

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2023-08-20 05:34:12,184: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-20 05:34:12,196: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-20 05:34:12,199: INFO: common: created directory at: artifacts]
[2023-08-20 05:34:12,199: INFO: common: created directory at: artifacts/model_evaluation]


C:\Users\Mikhnin\AppData\Local\Temp\ipykernel_21436\3695441855.py:60: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge') # загрузить метрику
100%|██████████| 5/5 [03:26<00:00, 41.22s/it]

[2023-08-20 05:37:48,237: INFO: rouge_scorer: Using default tokenizer.]
